In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
import os, sys
import numpy as np
import copy
import csv
import scipy.interpolate as itp
import math
import pandas as pd


In [ ]:
elem_file = "elems.txt"
elem_con = []
with open(elem_file) as nfile:
    for line in nfile:
        data = line.strip('\n').split(', ')
        elem_con.append([int(data[0]), int(data[1]), int(data[2]), int(data[3]), int(data[4]), int(data[5]), int(data[6]), int(data[7]), int(data[8]), int(data[9]), int(data[10])])

nodes_file = "nodes.txt"
node_loc = []
with open(nodes_file) as nfile:
    for line in nfile:
        data = line.strip('\n').split(',')  # sometimes you have to put space after comma
        node_loc.append([int(data[0]), float(data[1]), float(data[2]), float(data[3])])

set_file = "segmentElemSet.txt"
for n in range(1,6): # Number of loop to run
    globals()['slice%s_elems' % n] = []
    with open(set_file) as sfile:
        line = 'start'
        while line != '':
            line = sfile.readline()
            if ("slice%s"%n) in line:
                line = sfile.readline()
                while "*" not in line:
                    aa=(line.strip().split(','))
                    globals()['slice%s_elems' % n].append(aa)
                    line = sfile.readline()
                break
    globals()['slice%s_elems' % n] = [item for sublist in globals()['slice%s_elems' % n] for item in sublist]
    globals()['slice%s_elems' % n] = [item for item in globals()['slice%s_elems' % n] if item != '']
    globals()['slice%s_elems' % n] = [int(item) for item in globals()['slice%s_elems' % n]]

In [ ]:
# reading axes from orientation files
CRL_file = "CRL-1.txt"
CRL_axes = []
with open(CRL_file) as nfile:
    for line in nfile:
        data = line.strip('\n').split(',')
        CRL_axes.append([float(data[0]), float(data[1]), float(data[2]), float(data[3]), float(data[4]), float(data[5]), float(data[6]), float(data[7]), float(data[8]), float(data[9])])
CRL_axes = np.array(CRL_axes)

FNS_file = "FNS-1.txt"   # See text file of FNS, first coulmn is repeating
FNS_axes = []
with open(FNS_file) as nfile:
    for line in nfile:
        data = line.strip('\n').split(',')
        FNS_axes.append([float(data[0]), float(data[1]), float(data[2]), float(data[3]), float(data[4]), float(data[5]), float(data[6]), float(data[7]), float(data[8]), float(data[9])])
FNS_axes = np.array(FNS_axes)


In [ ]:
# Time incerement from abaqus sta file
time = [0]
line_no = 0

with open("PVloop-total-1.sta",'r') as sta_file:
    for line in sta_file:
        line_no += 1
        if line_no >= 6:
            try:
                data = line.split()
                if 'U' not in data[2]:
                    time.append(data[6])
            except:
                break

time = [float(item) for item in time]

data = {'numbers': time}
df = pd.DataFrame(data)

# find the closest number to a given value
given_value = 1.43          # End systole
idx_closest = (df['numbers']-given_value).abs().argsort()[0]


In [ ]:
examp = 1
dat_file =  "PVloop-total-" + str(examp) +".dat"
for m in range(1,6):
  # Find average polar strains on slice 1:
  marker = 0
  marks = []

  (globals()['slice%s_pt_strains_polar_C' % m]) = []
  (globals()['slice%s_pt_strains_polar_R' % m]) = []
  (globals()['slice%s_pt_strains_polar_L' % m]) = []

  (globals()['slice%s_pt_strains_polar_F' % m]) = []
  (globals()['slice%s_pt_strains_polar_N' % m]) = []
  (globals()['slice%s_pt_strains_polar_S' % m]) = []

  globals()['NElem_slice%s'%m] = len(globals()['slice%s_elems' % m]);

  with open(dat_file) as dfile:
      for line in dfile:
          marker += 1
          if ("SET SLICE%s"%m) in line:
              marks.append(marker)


  Q = np.zeros((3,3))
  for mark in marks:
      # read strains from last increment
      slice_elem_strains_polar = np.zeros([1,4])
      slice_elem_strains_polar_FNS = np.zeros([1,4])

      with open(dat_file) as dfile:
          for i in range(0,mark+4):
              dfile.readline()
          for n in range(0, globals()['NElem_slice%s'%m]):
              line = dfile.readline()
              data = line.split()

              strain = [[float(data[2]), float(data[5]), float(data[6])],
              [float(data[5]), float(data[3]), float(data[7])],
              [float(data[6]), float(data[7]), float(data[4])]]

              # fiber
              Q[0][0] = np.dot(FNS_axes[n,1:4], CRL_axes[n,1:4])
              Q[0][1] = np.dot(FNS_axes[n,1:4], CRL_axes[n,4:7])
              Q[0][2] = np.dot(FNS_axes[n,1:4], CRL_axes[n,7:10])

              # normal
              Q[1][0] = np.dot(FNS_axes[n,4:7], CRL_axes[n,1:4])
              Q[1][1] = np.dot(FNS_axes[n,4:7], CRL_axes[n,4:7])
              Q[1][2] = np.dot(FNS_axes[n,4:7], CRL_axes[n,7:10])

              # sheet
              Q[2][0] = np.dot(FNS_axes[n,7:10], CRL_axes[n,1:4])
              Q[2][1] = np.dot(FNS_axes[n,7:10], CRL_axes[n,4:7])
              Q[2][2] = np.dot(FNS_axes[n,7:10], CRL_axes[n,7:10])


              strain_tran1 = np.matmul(np.transpose(Q), strain)
              strain_tran = np.matmul(strain_tran1,Q)


              slice_elem_strains_polar = np.append(slice_elem_strains_polar,[[int(data[0]), float(strain_tran[0][0]), float(strain_tran[1][1]), float(strain_tran[2][2])]], axis=0)
              slice_elem_strains_polar_FNS = np.append(slice_elem_strains_polar_FNS,[[int(data[0]), float(data[2]), float(data[3]), float(data[4])]], axis=0)

      (globals()['slice%s_pt_strains_polar_C' % m]).append(slice_elem_strains_polar[1:,1])
      (globals()['slice%s_pt_strains_polar_R' % m]).append(slice_elem_strains_polar[1:,2])
      (globals()['slice%s_pt_strains_polar_L' % m]).append(slice_elem_strains_polar[1:,3])

      (globals()['slice%s_pt_strains_polar_F' % m]).append(slice_elem_strains_polar_FNS[1:,1])
      (globals()['slice%s_pt_strains_polar_N' % m]).append(slice_elem_strains_polar_FNS[1:,2])
      (globals()['slice%s_pt_strains_polar_S' % m]).append(slice_elem_strains_polar_FNS[1:,3])

      # create dataframes from the lists
      df1 = pd.DataFrame((globals()['slice%s_pt_strains_polar_C' % m])).T
      df2 = pd.DataFrame((globals()['slice%s_pt_strains_polar_R' % m])).T
      df3 = pd.DataFrame((globals()['slice%s_pt_strains_polar_L' % m])).T

      df4 = pd.DataFrame((globals()['slice%s_pt_strains_polar_F' % m])).T
      df5 = pd.DataFrame((globals()['slice%s_pt_strains_polar_N' % m])).T
      df6 = pd.DataFrame((globals()['slice%s_pt_strains_polar_S' % m])).T

      # create the output file name
      output_file_name = f"examp{examp}_slice_{m}.xlsx"
      # output_file_name = f"slice{m}.xlsx"

      # create an Excel writer object
      with pd.ExcelWriter(output_file_name) as writer:

      # create an Excel writer object
      # with pd.ExcelWriter('output%s.xlsx'%m) as writer:
          df1.to_excel(writer, sheet_name='Circum', index=False, header=False)
          df2.to_excel(writer, sheet_name='Radial', index=False, header=False)
          df3.to_excel(writer, sheet_name='Longit', index=False, header=False)
          df4.to_excel(writer, sheet_name='fiber', index=False, header=False)
          df5.to_excel(writer, sheet_name='normal', index=False, header=False)
          df6.to_excel(writer, sheet_name='sheet', index=False, header=False)

  data = {'Circum': (globals()['slice%s_pt_strains_polar_C' % m])[idx_closest],
          'Radial': (globals()['slice%s_pt_strains_polar_R' % m])[idx_closest],
          'Longit': (globals()['slice%s_pt_strains_polar_L' % m])[idx_closest],
          'fiber': (globals()['slice%s_pt_strains_polar_F' % m])[idx_closest],
          'normal': (globals()['slice%s_pt_strains_polar_N' % m])[idx_closest],
          'sheet': (globals()['slice%s_pt_strains_polar_S' % m])[idx_closest]}

  # create a Pandas dataframe from the dictionary for this iteration
  df = pd.DataFrame(data)

  # create the output file name
  output_file_name = f"examp_ES_CRLFNS_{examp}.xlsx"

  # check if the file exists before opening it in append mode
  if os.path.isfile(output_file_name):
      mode = 'a'
  else:
      mode = 'w'
  # create an Excel writer object
  with pd.ExcelWriter(output_file_name, mode=mode) as writer:

  # create an Excel writer object
      sheet_name = f'Slice{m}'
      df.to_excel(writer, sheet_name=sheet_name, index=False)
